# LDA + LLMs

En este notebook, exploraremos tres enfoques para organizar y entender grandes volúmenes de texto.

In [ ]:
!pip install -U datasets pyldavis spacy nltk

In [ ]:
import re
import unicodedata
from collections import Counter
from textwrap import shorten

import numpy as np
import pandas as pd

from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis
from IPython.display import display

# Stemming (Spanish)
from nltk.stem.snowball import SnowballStemmer
import warnings
warnings.filterwarnings("ignore")

## Tarea 1: Conectar con la API de OpenAI

Instrucciones detalladas: https://drive.google.com/file/d/1tvE9cA6lxK6OdgBD1-_5-Y14ObUHEmJ8/view

Una manera de cargar la api key en el entorno local y mantenerla escondida es con `getpass`

In [ ]:
import getpass
openai_key = getpass.getpass()

··········


In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=openai_key
)

response = client.responses.create(
    model="gpt-4.1-nano",
    input="¿Cuáles son las ventajas de LDA frente a un LLM para clasificación de texto?",
    store=True
)

In [ ]:
print(response)

Response(id='resp_02f109c412202de700697c033cbc588195a79d2b69af2349ee', created_at=1769734972.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-nano-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_02f109c412202de700697c033e32dc8195bdf9de98375cef61', content=[ResponseOutputText(annotations=[], text='Las ventajas de usar **LDA (Latent Dirichlet Allocation)** en comparación con **LLM (Modelos de Lenguaje Grandes)** para clasificación de texto, particularmente en ciertos contextos, son las siguientes:\n\n1. **Interpretable y transparente:**\n   - LDA genera temas que son conjuntos de palabras que pueden ser interpretados fácilmente, lo que facilita entender por qué un documento fue clasificado de cierta manera.\n   - Los LLM (como GPT o BERT) funcionan como "cajas negras", haciendo que la interpretación de las decisiones sea más difícil.\n\n2. **Menor requerimiento computacional y de recursos:**\n   - LDA es generalmente menos exige

In [ ]:
print(response.output_text)

Las ventajas de usar **LDA (Latent Dirichlet Allocation)** en comparación con **LLM (Modelos de Lenguaje Grandes)** para clasificación de texto, particularmente en ciertos contextos, son las siguientes:

1. **Interpretable y transparente:**
   - LDA genera temas que son conjuntos de palabras que pueden ser interpretados fácilmente, lo que facilita entender por qué un documento fue clasificado de cierta manera.
   - Los LLM (como GPT o BERT) funcionan como "cajas negras", haciendo que la interpretación de las decisiones sea más difícil.

2. **Menor requerimiento computacional y de recursos:**
   - LDA es generalmente menos exigente en términos de hardware y tiempo de entrenamiento comparado con LLMs, que requieren GPUs potentes y largos períodos de entrenamiento o ajuste fino.

3. **Entrenamiento más simple y rápido:**
   - La implementación y ajuste de LDA suele ser más sencillo y requiere menos datos para obtener buenos resultados, lo que lo hace adecuado en escenarios con recursos li

Puedes encontrar una lista de todos los modelos en: https://platform.openai.com/docs/models

**¡Cuidado!** Hay modelos que son significativamente más caros de utilizar.

Recomendamos usar: https://platform.openai.com/docs/models/gpt-4.1-nano

In [ ]:
# Si store=True, la respuesta se guardará en los servidores de OpenAI con cierto ID
print(response.id)

resp_02f109c412202de700697c033cbc588195a79d2b69af2349ee


## Tarea 2: Entrenar LDA sobre el dataset `OpenAssistant`

In [ ]:
# -----------------------
# 1) Load dataset
# -----------------------
ds = load_dataset("OpenAssistant/oasst1")
df = pd.concat([ds["train"].to_pandas(), ds["validation"].to_pandas()],
               ignore_index=True)

In [ ]:
# Para replicabilidad
RANDOM_SEED = 42

In [ ]:
# -----------------------
# 2) Filter: Spanish root prompts from the user
# -----------------------
USER_ROLES = {"user", "prompter", "human"}

df = df[df["role"].astype(str).str.lower().isin(USER_ROLES)]
df = df[df["parent_id"].isna()]                 # root message in the thread
df = df[df["deleted"] == False]
df = df[df["synthetic"] == False]
df = df[df["lang"].astype(str).str.lower().str.startswith("es")]

docs_raw = df["text"].astype(str).tolist()
print("Docs (raw):", len(docs_raw))

Docs (raw): 3910


In [ ]:
# -----------------------
# 3) Preprocessing (standard NLP pipeline + stemming)
#    clean -> normalize -> tokenize -> stopwords -> stem -> re-join
# -----------------------
def strip_accents(s: str) -> str:
    return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

STOPWORDS_ES = {
    # common Spanish
    "de","la","que","el","en","y","a","los","del","se","las","por","un","para","con","no",
    "una","su","al","lo","como","mas","pero","sus","le","ya","o","este","si","porque",
    "esta","entre","cuando","muy","sin","sobre","tambien","me","hasta","hay","donde",
    "quien","desde","todo","nos","durante","todos","uno","les","ni","contra","otros",

    # question/prompt words (important for chatbot prompts)
    "que","como","cual","cuales","quien","donde","cuando","cuanto","cuantos","por","porque",

    # prompt fluff
    "hola","buenas","gracias","porfavor","favor",
    "puedes","podrias","ayudame","necesito","dime","explica","explicame","describe","resume",
    "haz","dame","escribe","crea","genera",

    # common high-frequency verbs/forms
    "ser","estar","tener","hacer","poder",
    "soy","eres","es","son","estoy","esta","estan","tengo","tiene","tienen","quiero","puedo",
}

# Normalize stopwords in the same way we normalize text
STOPWORDS_ES = {strip_accents(w.lower()) for w in STOPWORDS_ES}

# Spanish stemmer (simple + no extra downloads)
stemmer = SnowballStemmer("spanish")

def preprocess(text: str) -> str:
    # A) CLEANING (remove obvious noise)
    text = (text or "").lower()
    text = re.sub(r"https?://\S+|www\.\S+", " ", text)  # remove URLs

    # B) NORMALIZATION (accent folding)
    text = strip_accents(text)  # "qué" -> "que"

    # C) TOKENIZATION (letters only; after accent stripping)
    tokens = re.findall(r"[a-zñ]+", text)

    # D) FILTERING (stopwords + short tokens)
    tokens = [t for t in tokens if len(t) >= 3 and t not in STOPWORDS_ES]

    # E) STEMMING (reduce words to their root)
    tokens = [stemmer.stem(t) for t in tokens]

    # Return a string for CountVectorizer
    return " ".join(tokens)

docs_clean = [preprocess(t) for t in docs_raw]

# Keep alignment between raw and clean by filtering pairs together
pairs = [(r, c) for r, c in zip(docs_raw, docs_clean) if c.strip()]
docs_raw, docs_clean = zip(*pairs) if pairs else ([], [])
docs_raw, docs_clean = list(docs_raw), list(docs_clean)

print("Docs (clean, non-empty):", len(docs_clean))
print("Top tokens after preprocessing:", Counter(" ".join(docs_clean).split()).most_common(20))


Docs (clean, non-empty): 3885
Top tokens after preprocessing: [('mejor', 245), ('pued', 240), ('cre', 221), ('list', 204), ('program', 197), ('diferent', 179), ('deb', 177), ('algun', 164), ('funcion', 152), ('pas', 142), ('ayud', 137), ('dec', 136), ('explic', 133), ('cad', 133), ('python', 131), ('palabr', 130), ('form', 127), ('sistem', 124), ('cuent', 123), ('siguient', 123)]


In [ ]:
# -----------------------
# 4) Bag-of-words
# -----------------------
vectorizer = CountVectorizer(max_df=0.9, min_df=2, ngram_range=(1, 2))
X = vectorizer.fit_transform(docs_clean)
vocab = np.array(vectorizer.get_feature_names_out())
print("Vocab size:", len(vocab))

Vocab size: 5129


In [ ]:
# -----------------------
# 5) Fit LDA
# -----------------------
n_topics = 12
lda = LatentDirichletAllocation(
    n_components=n_topics,
    learning_method="batch",
    random_state=RANDOM_SEED,
    max_iter=25
)
lda.fit(X)

LatentDirichletAllocation(max_iter=25, n_components=12, random_state=42)

In [ ]:
# -----------------------
# 6) Topics: print + DataFrame of top words per topic
# -----------------------
def show_topics(model, vocab, top_n=12):
    for k, weights in enumerate(model.components_):
        top_idx = np.argsort(weights)[::-1][:top_n]
        top_terms = [vocab[i] for i in top_idx]
        print(f"Topic {k}: {', '.join(top_terms)}")

def topics_dataframe(model, vocab, top_n=12):
    rows = []
    for topic_id, weights in enumerate(model.components_):
        top_idx = np.argsort(weights)[::-1][:top_n]
        for rank, i in enumerate(top_idx, start=1):
            rows.append({
                "topic_id": topic_id,
                "rank": rank,
                "term": vocab[i],
                "weight": float(weights[i]),
            })
    return pd.DataFrame(rows)

show_topics(lda, vocab, top_n=12)

df_topics = topics_dataframe(lda, vocab, top_n=12)


Topic 0: inteligent, artificial, inteligent artificial, color, pued, gener, mund, ley, public, estad, deb, algun
Topic 1: pued, cuent, vid, sistem, person, sea, ten, estas, histori, algun, dec, mejor
Topic 2: recet, prepar, ayud, chist, dec, rim, poem, cocin, ingredient, import, bas, cas
Topic 3: nin, anos, program, aprend, dat, web, pagin, maner, palabr, libr, nin anos, python
Topic 4: fue, funcion, hazm, list, pas, explic, medi, resum, utiliz, cas, gener, cre
Topic 5: sistem, pas, relacion, funcion, libr, estudi, calcul, cre, deb, list, pued, form
Topic 6: open, open assistant, assistant, mejor, algun, salud, pais, list, ventaj, ayud, dar, pued
Topic 7: list, mejor, cre, hac, tip, vide, youtub, cad, personaj, ano, histori, cuant
Topic 8: program, diferent, lenguaj, python, lenguaj program, sab, gustari, codig, funcion, proces, ejempl, client
Topic 9: palabr, sol, siguient, algun, conoc, planet, diferent, the, espanol, utiliz, hombr, ayud
Topic 10: mejor, cuent, mund, cre, numer, pais

In [ ]:
df_topics

,topic_id,rank,term,weight
0,0,1,inteligent,74.834600
1,0,2,artificial,72.907482
2,0,3,inteligent artificial,68.845213
3,0,4,color,49.352015
4,0,5,pued,37.212863
...,...,...,...,...
139,11,8,jug,21.667280
140,11,9,deb,21.395097
141,11,10,biciclet,21.074805
142,11,11,sab,20.620359


In [ ]:
# -----------------------
# 7) Documents: DataFrame with topic assignment + probabilities
# -----------------------
doc_topic = lda.transform(X)
top_topic = doc_topic.argmax(axis=1)
top_conf = doc_topic.max(axis=1)

df_docs = pd.DataFrame({
    "doc_id": np.arange(len(docs_raw)),
    "text_raw": [shorten(t, width=160, placeholder="…") for t in docs_raw],
    "text_clean": [shorten(t, width=160, placeholder="…") for t in docs_clean],
    "top_topic": top_topic,
    "top_topic_conf": np.round(top_conf, 3),
})

for k in range(n_topics):
    df_docs[f"topic_{k}_prob"] = np.round(doc_topic[:, k], 3)

df_docs

,doc_id,text_raw,text_clean,top_topic,top_topic_conf,topic_0_prob,topic_1_prob,topic_2_prob,topic_3_prob,topic_4_prob,topic_5_prob,topic_6_prob,topic_7_prob,topic_8_prob,topic_9_prob,topic_10_prob,topic_11_prob
0,0,¿CUales son las etapas del desarrollo y en qué...,etap desarroll consist segun piaget,9,0.608,0.017,0.017,0.017,0.017,0.017,0.017,0.017,0.017,0.017,0.608,0.017,0.226
1,1,Método del Perceptrón biclásico: definición y ...,metod perceptron biclas definicion variant met...,8,0.935,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.935,0.006,0.006,0.006
2,2,Estoy pensando en crear un videojuego pero no ...,pens cre videojueg segur motor jueg utiliz bus...,7,0.816,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.816,0.171,0.001,0.001,0.001
3,3,¿Qué son los gases nobles?,gas nobl,4,0.542,0.042,0.042,0.042,0.042,0.542,0.042,0.042,0.042,0.042,0.042,0.042,0.042
4,4,que actividades se supone que son comunes en u...,activ supon comun person anos,3,0.869,0.012,0.012,0.012,0.869,0.012,0.012,0.012,0.012,0.012,0.012,0.012,0.012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3880,3880,Necesito que hagas de guía de viajes. Viajaré ...,hag gui viaj viajar madr sug diferent monument...,10,0.608,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.138,0.005,0.608,0.212
3881,3881,"Dime cuál es el resultado de sumar 2 y 2, y ju...",result sum justif respuest,7,0.869,0.012,0.012,0.012,0.012,0.012,0.012,0.012,0.869,0.012,0.012,0.012,0.012
3882,3882,saludame,saludam,0,0.083,0.083,0.083,0.083,0.083,0.083,0.083,0.083,0.083,0.083,0.083,0.083,0.083
3883,3883,Qué reuniones tengo en mi agenda para esta sem...,reunion agend seman,11,0.694,0.028,0.028,0.028,0.028,0.028,0.028,0.028,0.028,0.028,0.028,0.028,0.694


In [ ]:
# -----------------------
# 8) pyLDAvis
# -----------------------
pyLDAvis.enable_notebook()

topic_term_dists = lda.components_ / lda.components_.sum(axis=1)[:, None]
doc_topic_dists = doc_topic
doc_lengths = np.asarray(X.sum(axis=1)).ravel()
term_frequency = np.asarray(X.sum(axis=0)).ravel()

vis = pyLDAvis.prepare(
    topic_term_dists=topic_term_dists,
    doc_topic_dists=doc_topic_dists,
    doc_lengths=doc_lengths,
    vocab=vocab,
    term_frequency=term_frequency,
    sort_topics=False
)

display(vis)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.059190  0.009529       1        1   8.809750
1      0.033642 -0.013803       2        1   9.683859
2     -0.112279 -0.177260       3        1   5.731590
3      0.157450 -0.047317       4        1   8.002109
4      0.065541  0.081487       5        1   7.224614
5      0.016403 -0.018481       6        1   7.716887
6      0.012355 -0.048464       7        1   7.028729
7     -0.044845  0.002686       8        1   7.044029
8      0.118317  0.052136       9        1  11.293724
9     -0.030864 -0.063618      10        1   8.845844
10    -0.010151  0.064475      11        1  10.786919
11    -0.146379  0.158629      12        1   7.831945, topic_info=                       Term        Freq       Total Category  logprob  loglift
3882                program  176.000000  176.000000  Default  30.0000  30.0000
369              artificial   66.000000   66.000000  Default  29.0000  29.0000
2674                   jueg   88.000000   88.000000  Default  28.0000  28.0000
2595             inteligent   73.000000   73.000000  Default  27.0000  27.0000
2597  inteligent artificial   62.000000   62.000000  Default  26.0000  26.0000
...                     ...         ...         ...      ...      ...      ...
1294                    dec   12.458808  120.842858  Topic12  -5.5183   0.2749
1083                    cre   12.947039  196.610300  Topic12  -5.4798  -0.1734
3518                    pas   11.513537  126.721448  Topic12  -5.5972   0.1485
1170                  cuent   11.268486  111.013771  Topic12  -5.6187   0.2593
2404                histori   10.713448  102.969813  Topic12  -5.6692   0.2840

[826 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1        12  0.906729  abandon
11        2  0.093079    abuel
11        3  0.093079    abuel
11       11  0.651555    abuel
11       12  0.093079    abuel
...     ...       ...      ...
5114     11  0.111744  windows
5116      2  0.116946     word
5116      3  0.818625     word
5124      8  0.827936   youtub
5124     10  0.157702   youtub

[2252 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

## Tarea 3: Clasificar los tópicos de `OpenAssistant` con ChatGPT

In [ ]:
system_msg = (
    "Eres un generador de etiquetas de temas en español. "
    "Tu única salida debe ser UNA SOLA LÍNEA con la etiqueta del tema, en minúsculas, "
    "máximo 3 palabras, sin puntuación final, sin comillas y sin explicaciones. "
    "Ejemplos válidos: inteligencia artificial, química inorgánica, deportes."
)

examples = (
    "Ejemplos (solo referencia, NO repitas estos ejemplos en la salida):\n"
    "Texto: '¿Qué es un perceptrón y cómo se entrena una red neuronal?' -> aprendizaje automático\n"
    "Texto: 'Propiedades y usos del helio y el neón' -> gases nobles\n"
)

# Llamemos al LLM en los primeros 100 documentos. ¡Ojo con no quemar tu presupuesto!
num_docs_to_label = 100
doc_label_rows = []

for i in range(num_docs_to_label):
    doc_row = df_docs.iloc[i]
    doc_id = int(doc_row['doc_id']) if 'doc_id' in doc_row.index else i
    text_raw = str(doc_row.get('text_raw', '')).strip()

    user_msg = (
        f"{examples}\n"
        "Genera UNA etiqueta corta (1-3 palabras) en español que resuma el siguiente texto. "
        "Devuelve SOLO la etiqueta en minúsculas, sin comillas ni texto adicional.\n\n"
        f"Texto: {text_raw}"
    )

    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg},
        ],
        temperature=0.1
    )

    raw_label = response.choices[0].message.content.strip()

    # Minimal normalization: remove surrounding quotes/backticks, trailing punctuation, lowercased
    label = re.sub(r'^[`"\']+|[`"\']+$', '', raw_label).strip().lower()
    label = re.sub(r'[ .,:;!¡?¿]+$', '', label)

    doc_label_rows.append({"doc_id": doc_id, "llm_doc_label": label})

# Create a DataFrame of LLM document labels and merge back into df_docs
df_llm_doc_labels = pd.DataFrame(doc_label_rows)

df_docs_chatgpt = pd.merge(df_docs, df_llm_doc_labels, on='doc_id', how='left')

# Show the labeled subset
print("First labeled documents (doc_id, top_topic, llm_doc_label):")
display(df_docs_chatgpt.loc[:num_docs_to_label - 1, ["doc_id", "text_raw", "top_topic", "top_topic_conf", "llm_doc_label"]])

First labeled documents (doc_id, top_topic, llm_doc_label):


,doc_id,text_raw,top_topic,top_topic_conf,llm_doc_label
0,0,¿CUales son las etapas del desarrollo y en qué...,9,0.608,desarrollo cognitivo
1,1,Método del Perceptrón biclásico: definición y ...,8,0.935,aprendizaje automático
2,2,Estoy pensando en crear un videojuego pero no ...,7,0.816,desarrollo de videojuegos
3,3,¿Qué son los gases nobles?,4,0.542,gases nobles
4,4,que actividades se supone que son comunes en u...,3,0.869,vida adulta
...,...,...,...,...,...
95,95,¿Me puedes ayudar a predecir los resultados de...,7,0.968,predicción deportiva
96,96,"Hola, podrías decirme como estará el clima en ...",3,0.885,pronóstico del tiempo
97,97,"¿Cuánto costaba (aproximadamente, en dólares) ...",4,0.869,vehículos eléctricos
98,98,De manera resumida explica que pasaria si una ...,3,0.657,leyes de la física


In [ ]:
# ¿Cuántos tópicos se generaron?
df_docs_chatgpt.llm_doc_label.nunique()

94

## Tarea 4: Clasificar los tópicos de `OpenAssistant` con ChatGPT y LDA

Se puede mejorar el procesos groundeando al LLM con los tópicos generados por LDA.

In [ ]:
system_msg = (
    "Eres un generador de etiquetas de temas en español. "
    "Tu única salida debe ser UNA SOLA LÍNEA con la etiqueta del tema, en minúsculas, "
    "máximo 3 palabras, sin puntuación final, sin comillas y sin explicaciones."
)

llm_labels = []
unique_topic_ids = df_topics['topic_id'].unique()

for topic_id in unique_topic_ids:
    topic_df = df_topics[df_topics['topic_id'] == topic_id].copy()
    topic_keywords = topic_df['term'].astype(str).tolist()[:50]
    keyword_str = ", ".join(topic_keywords)

    user_msg = (
        "Genera UNA etiqueta corta (1-3 palabras) en español que resuma estas palabras clave. "
        "Devuelve SOLO la etiqueta en minúsculas, sin comillas ni texto adicional.\n\n"
        f"Palabras clave: {keyword_str}"
    )

    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg},
        ],
        temperature=0.1
    )

    raw_label = response.choices[0].message.content.strip()
    label = re.sub(r'^[`"\']+|[`"\']+$', '', raw_label).strip().lower()
    label = re.sub(r'[ .,:;!¡?¿]+$', '', label)
    llm_labels.append({"topic_id": topic_id, "llm_label": label})

df_llm_labels = pd.DataFrame(llm_labels)

df_topics = pd.merge(df_topics, df_llm_labels, on="topic_id", how="left")

topic_label_map = df_llm_labels.set_index("topic_id")["llm_label"]
df_docs["top_topic_label"] = df_docs["top_topic"].map(topic_label_map)

print("df_topics sample with llm_label:")
display(df_topics)

df_topics sample with llm_label:


,topic_id,rank,term,weight,llm_label
0,0,1,inteligent,74.834600,tecnología y ley
1,0,2,artificial,72.907482,tecnología y ley
2,0,3,inteligent artificial,68.845213,tecnología y ley
3,0,4,color,49.352015,tecnología y ley
4,0,5,pued,37.212863,tecnología y ley
...,...,...,...,...,...
139,11,8,jug,21.667280,deportes y tiempo
140,11,9,deb,21.395097,deportes y tiempo
141,11,10,biciclet,21.074805,deportes y tiempo
142,11,11,sab,20.620359,deportes y tiempo


In [ ]:
system_msg = (
    "Eres un generador de etiquetas de temas en español. "
    "Tu única salida debe ser UNA SOLA LÍNEA con la etiqueta del tema, "
    "en minúsculas, máximo 3 palabras, sin puntuación final, sin comillas y sin explicaciones."
)

# prepare topic metadata: one label per topic_id + up to 10 unique terms
topic_terms = (
    df_topics.groupby("topic_id")["term"]
    .apply(lambda terms: list(dict.fromkeys([str(t).strip() for t in terms if str(t).strip()]))[:10])
    .to_dict()
)
topic_labels = df_topics.drop_duplicates("topic_id").set_index("topic_id")["llm_label"].to_dict()
all_topic_ids = list(topic_labels.keys())

num_docs = 100
rows = []

candidates = [f"{topic_labels.get(t,'tema_'+str(t))} — {topic_terms.get(t,'')}" for t in all_topic_ids]
candidates_text = "\n".join(f"{j+1}) {c}" for j, c in enumerate(candidates))
for i in range(num_docs):
    r = df_docs.iloc[i]
    doc_id = int(r["doc_id"]) if "doc_id" in r.index else int(i)
    text = str(r.get("text_raw", "")).strip()

    user_msg = (
        "A continuación tienes una lista de etiquetas candidatas (etiqueta — palabras clave):\n"
        f"{candidates_text}\n\n"
        "Texto a etiquetar:\n"
        f"{text}\n\n"
        "Elige UNA de esas etiquetas y RESPONDE SOLO con la etiqueta final en minúsculas, sin comillas ni texto adicional."
    )

    resp = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[{"role": "system", "content": system_msg}, {"role": "user", "content": user_msg}],
        temperature=0.1,
    )
    raw = resp.choices[0].message.content.strip()
    label = re.sub(r'^[`"\']+|[`"\']+$', '', raw).strip().lower()
    label = re.sub(r'[ .,:;!¡?¿]+$', '', label)
    rows.append({"doc_id": doc_id, "grounded_label": label})

df_llm = pd.DataFrame(rows)
df_result = df_docs.merge(df_llm, on="doc_id", how="left")
display(df_result.loc[: num_docs - 1, ["doc_id", "text_raw", "grounded_label"]])

,doc_id,text_raw,grounded_label
0,0,¿CUales son las etapas del desarrollo y en qué...,conocimiento planetario
1,1,Método del Perceptrón biclásico: definición y ...,funciones y usos
2,2,Estoy pensando en crear un videojuego pero no ...,programacion y lenguajes
3,3,¿Qué son los gases nobles?,conocimiento planetario
4,4,que actividades se supone que son comunes en u...,relatos personales
...,...,...,...
95,95,¿Me puedes ayudar a predecir los resultados de...,deportes y tiempo
96,96,"Hola, podrías decirme como estará el clima en ...",deportes y tiempo
97,97,"¿Cuánto costaba (aproximadamente, en dólares) ...",ranking mundial
98,98,De manera resumida explica que pasaria si una ...,funciones y usos


In [ ]:
pd.merge(df_result[["doc_id", "text_raw", "grounded_label"]], df_docs[["doc_id", "top_topic_label"]], on="doc_id").head(15)

,doc_id,text_raw,grounded_label,top_topic_label
0,0,¿CUales son las etapas del desarrollo y en qué...,conocimiento planetario,conocimiento planetario
1,1,Método del Perceptrón biclásico: definición y ...,funciones y usos,programacion y lenguajes
2,2,Estoy pensando en crear un videojuego pero no ...,programacion y lenguajes,contenido digital
3,3,¿Qué son los gases nobles?,conocimiento planetario,funciones y usos
4,4,que actividades se supone que son comunes en u...,relatos personales,programacion infantil
5,5,Explícame paso a paso la receta para hacer una...,recetas y cocina,recetas y cocina
6,6,Dime la definición de Psicópata.,relatos personales,contenido digital
7,7,¿Qué similitudes hay entre la música Barroca y...,conocimiento planetario,programacion y lenguajes
8,8,¿Cuál es la mejor manera de aprender diseño el...,funciones y usos,ranking mundial
9,9,Escribe un texto argumentativo sobre la música...,contenido digital,deportes y tiempo


## Tarea 5: Reflexionar sobre las siguientes cuestiones

## Conclusiones y Reflexión
Cada uno de los métodos que vimos para clasificación de texto tiene un balance distinto entre **autonomía**, **costo** e **interpretabilidad**:

1. LDA (Latent Dirichlet Allocation): Modelo probabilístico que descubre tópicos ocultos en el dataset.

2. LLM (Large Language Model): Uso vía API de un LLM para clasificación de texto.

3. LLM + LDA: Enfoque híbrido donde el LDA agrupa los documentos y el LLM etiqueta los tópicos descubiertos.

Reflexiona y responde las siguientes cuestiones:

- **Precisión vs. Semántica:** ¿Qué desventaja presenta el LDA (1) en cuanto a los nombres de los tópicos frente a los métodos (2) y (3)?

- **Análisis de Costos:** Ordena los tres métodos de mayor a menor costo financiero (dinero invertido en tokens de API) si tuvieras que procesar un corpus de 10 millones de documentos.

- **Recursos Locales:** ¿Cuál es el método más exigente a nivel de cómputo local (CPU/RAM)? Supón que tenemos acceso libre al API de OpenAI.